In [1]:
import os
import time
import lda
import random
import argparse
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import bokeh.plotting as bp
from bokeh.plotting import save
from bokeh.models import HoverTool
import json

In [2]:
#Preprocessing with Emoticons
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    #r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [3]:
#Counting Terms
import operator 
import json
from collections import Counter
#Removing Stop Words (spanish)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import bigrams 
from collections import defaultdict
import string

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.moses import MosesDetokenizer



[nltk_data] Downloading package stopwords to C:\Users\Sara
[nltk_data]     Palacios\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
set(stopwords.words('spanish'))
nltk.download('perluniprops')

[nltk_data] Downloading package perluniprops to C:\Users\Sara
[nltk_data]     Palacios\AppData\Roaming\nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!


True

In [5]:
punctuation = list(string.punctuation)
stop = stopwords.words('spanish') + punctuation + ['rt', 'via', '¡','…','¿','http','https','/','//', 'co', 't.co'] + stopwords.words('english')

In [132]:
#Se cambia el nombre del archivo
with open('GoogleColombia.json', 'r', encoding='utf8') as f:
    count_all = Counter()
    count_stop = Counter()
    count_hash = Counter()
    count_bigrams = Counter()
    documents = []
    original_tweets = []
    for line in f:
        tweet = json.loads(line)
        # Create a list with tweets
        original_tweets.append(tweet['text'])
        tweet_tokens = word_tokenize(tweet['text'].lower()) #Method of Tokenization 1
        #tweet_tokens =  preprocess(tweet['text'],True) #Method of Tokenization 2
        filtered_sentence = [w for w in tweet_tokens if not w in stop]
        documents.append(filtered_sentence)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'],True)]
        terms_stop = [term for term in preprocess(tweet['text'],True) if term not in stop]
        # Count hashtags only
        terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
        # Count bigrams
        terms_bigram = bigrams(terms_stop)
        # Update the counter
        count_all.update(terms_all)
        count_stop.update(terms_stop)
        count_hash.update(terms_hash)
        count_bigrams.update(terms_bigram)

In [133]:
 # Print the first 5 most frequent words
print(count_all.most_common(5))
print(count_stop.most_common(5))
print(count_hash.most_common(5))
print(count_bigrams.most_common(5))

[('/', 6955), ('.', 5374), ('https', 3527), (':/', 3505), ('t', 3493)]
[(':/', 3505), ('ón', 595), ('google', 367), ('️', 305), ('3', 276)]
[('#GoogleDoodle', 85), ('#GoogleMaps', 85), ('#DesafíoGoogleOrg', 77), ('#Google', 76), ('#GarageDigital', 68)]
[(('️', ':/'), 111), (('➡', '️'), 97), (('informaci', 'ón'), 95), (('est', 'á'), 92), (('➜', ':/'), 90)]


In [134]:
detokenizer = MosesDetokenizer()
tweets = []
for doc in documents:
    tweets.append(detokenizer.detokenize(doc, return_str=True))
print(tweets)

['|￣￣￣￣￣￣￣￣￣￣￣| ¡feliz puente |＿＿＿＿＿＿＿＿＿＿＿| •◡• /… //t.co/bjdpgcytgu', 'googleplaytime 2018 🇧🇷 evento llevó cabo ayer saopaulo convoca cientos desarrolladores p… //t.co/yzip5f97b1', 'google trabajamos garantizar seguro internet 🔐📱¡conoce visitando centro seguri… //t.co/amdajucbal', 'hoy saopaulo vive cuarta edición playtimelatam encuentro desarrolladores apps juegos 🎮📱en… //t.co/qczwvik2xz', 'día cappuccino ☕️😋 disfruta delicioso café colombiano 🇨🇴 ¡pregúntale asistentedegoogle //t.co/f7husqwqcq', 'dramaloverco ¡hola sentimos inconvenientes primero debes acceder cuent… //t.co/nvameuauby', "preguntados sabe bien lucianoluxios decir ``''😊 ¿nos cuentas si funciona", 'obra arte usaron 50.000 piezas plástico 🛍🌊 proyecto googlearts descubres inici… //t.co/uuk46cmjkr', 'asistentedegoogle podrás jugar preguntados experiencia divertida educativa toda familia 👪🙌 pí… //t.co/6lnm6tvrcj', 'colnodo ¡felicitaciones 😊', 'proyecto googlearts oportunidad explorar conocer diverso patrimonio cultura… //t.

In [135]:
no_features = 10000
# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features)
tfidf = tfidf_vectorizer.fit_transform(tweets)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()


In [136]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features)
tf = tf_vectorizer.fit_transform(tweets)
tf_feature_names = tf_vectorizer.get_feature_names()

In [137]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [138]:
#Aca se coloca el numero de temas que se quiere extraer
no_topics = 5

# Run NMF
nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf) #documents matrix (W)
nmf_H = nmf_model.components_ #topics matrix (H) 

In [139]:
n_iter = 5

# Run LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=n_iter, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [140]:
#Labeling Topics
topic_summaries = []
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic %d:" % (topic_idx))
        topic_words = " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
        topic_summaries.append(topic_words) # append!
        print(topic_words)
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print("Tweet %d:" % (doc_index) + documents[doc_index])

In [141]:

no_top_words = 5 #tiene que coincidir con los topics
no_top_documents = 4 #tweets

In [142]:
#Results for NMF model
display_topics(nmf_H, nmf_W, tfidf_feature_names, original_tweets, no_top_words, no_top_documents)

Topic 0:
co hoy youtube puedes googledoodle
Tweet 2289:Todo es "cuestión de percepción" ▶ https://t.co/yKwsmIDL3H. @enchufetv https://t.co/xjLkkwP84Q
Tweet 1907:https://t.co/xNxyLq0cCN
Tweet 2389:https://t.co/daCs5ouuAI
Tweet 2299:#TBT Cuando esto apenas empezaba... https://t.co/e279tKVamV
Topic 1:
gracias compartir cd_briganet carlos semana
Tweet 63:@MuseolaTertulia ¡Gracias por compartir!
Tweet 251:@CCIT_ 👏😎 ¡Gracias por compartir!
Tweet 62:@SantiagoDeCali_ Gracias por compartir 😉
Tweet 142:@ChiariJurgen 🙌😎 ¡Gracias por compartir!
Topic 2:
google co colombia cultural cafetero
Tweet 2052:Explora el paisaje cultural cafetero de Colombia en Google Street View: https://t.co/7fnne9IFTl. https://t.co/uYNw91VNiY
Tweet 2123:Explora el paisaje cultural cafetero de Colombia en Google Street View: https://t.co/7fnnea0hhV. https://t.co/9dRi3cjoQI
Tweet 2129:Explora el paisaje cultural cafetero de Colombia en Google Street View: https://t.co/7fnnea0hhV. https://t.co/OZgAx28Yqj
Tweet 2042:Explora 

In [143]:
#Results for LDA model
#es el mejor
display_topics(lda_H, lda_W, tf_feature_names, original_tweets, no_top_words, no_top_documents)

Topic 0:
co google puedes aprende si
Tweet 2456:“Toc Toc”. Con Google Duo mira quién llama antes de contestar. Disponible en Android https://t.co/6Sq7aMXzCW o iOS… https://t.co/DejSCKWkch
Tweet 2448:“Toc Toc”. Con Google Duo mira quién llama antes de contestar. Disponible en Android https://t.co/6Sq7aMXzCW o iOS… https://t.co/p9UfoTVinw
Tweet 2436:“Toc Toc”. Con Google Duo mira quién llama antes de contestar. Disponible en Android https://t.co/6Sq7aMXzCW o iOS… https://t.co/OjvhUafzRo
Tweet 90:El Programa de Pasantías de Google sigue abierto. Si vas a aplicar 👇

✅ Se tú mismo. Elige un rol que coincida con… https://t.co/MkdkEocMNX
Topic 1:
nueva googlecolombia co ghelp expertos
Tweet 1083:¿Tienes alguna duda o necesitas ayuda? 🤔 Recuerda incluir el hashtag #gHelp para que los expertos te puedan apoyar. https://t.co/5VL9EzHW3L
Tweet 2222:¿Tienes preguntas acerca de productos Google? Usa el hashtag #gHelp para recibir ayuda de usuarios expertos. https://t.co/6pLEMtvuo0
Tweet 2712:¿Tienes

In [144]:
#Visualizing with Dimensionality Reduction

from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [38]:
#Visualzing the models with pyLDAvis
pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer)

C:\Users\Sara Palacios\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      32.482588        1       1 -0.161728  0.039987
2      27.747325        1       2 -0.222233 -0.094089
0      15.939483        1       3  0.083150  0.133208
4      14.746966        1       4  0.107037  0.117484
3       9.083638        1       5  0.193774 -0.196590, topic_info=     Category         Freq                Term        Total  loglift  logprob
term                                                                         
450   Default  2057.000000                  co  2057.000000  30.0000  30.0000
761   Default   614.000000                  dm   614.000000  29.0000  29.0000
991   Default   349.000000               favor   349.000000  28.0000  28.0000
927   Default   257.000000           esperamos   257.000000  27.0000  27.0000
617   Default   448.000000           cuéntanos   448.000000  26.0000  26.0000
1112  Default   203.000000               gusto   203.000000  25.0000  25.0000
1643  Default   111.000000             momento   111.000000  24.0000  24.0000
2007  Default   219.000000              puedes   219.000000  23.0000  23.0000
2091  Default   151.000000               redes   151.000000  22.0000  22.0000
1091  Default   153.000000             gracias   153.000000  21.0000  21.0000
2184  Default   147.000000           saludarte   147.000000  20.0000  20.0000
535   Default   217.000000            contamos   217.000000  19.0000  19.0000
2538  Default   298.000000                 vía   298.000000  18.0000  18.0000
2185  Default   309.000000             saludos   309.000000  17.0000  17.0000
318   Default   111.000000           brindarte   111.000000  16.0000  16.0000
883   Default   133.000000            envíanos   133.000000  15.0000  15.0000
2027  Default   307.000000            queremos   307.000000  14.0000  14.0000
787   Default    78.000000                días    78.000000  13.0000  13.0000
1154  Default  1553.000000                hola  1553.000000  12.0000  12.0000
1760  Default   120.000000              número   120.000000  11.0000  11.0000
2242  Default   385.000000                  si   385.000000  10.0000  10.0000
327   Default    68.000000              buenos    68.000000   9.0000   9.0000
1128  Default   123.000000               hacer   123.000000   8.0000   8.0000
367   Default    75.000000             canales    75.000000   7.0000   7.0000
1926  Default   176.000000             podemos   176.000000   6.0000   6.0000
282   Default    81.000000         bancolombia    81.000000   5.0000   5.0000
1883  Default    84.000000          percepción    84.000000   4.0000   4.0000
708   Default    82.000000            detalles    82.000000   3.0000   3.0000
1798  Default    42.000000              opción    42.000000   2.0000   2.0000
968   Default    81.000000        experiencias    81.000000   1.0000   1.0000
...       ...          ...                 ...          ...      ...      ...
1      Topic5    11.780999                 000    12.578776   2.3332  -5.4578
683    Topic5    10.674625              demora    11.437653   2.3297  -5.5564
955    Topic5    11.085869              evitar    11.940096   2.3245  -5.5186
1129   Topic5    10.033140             hacerlo    10.845942   2.3208  -5.6184
1570   Topic5     9.071912             mascota     9.842309   2.3172  -5.7191
98     Topic5     8.775883                agro     9.542810   2.3149  -5.7523
2519   Topic5    10.182449               video    11.088400   2.3135  -5.6036
2368   Topic5     8.938620        telemedellin     9.781126   2.3086  -5.7339
1180   Topic5     8.346353            impuesto     9.137067   2.3082  -5.8024
1781   Topic5     8.928325  ofertasbancolombia     9.779316   2.3077  -5.7350
1158   Topic5     8.112577            horarios     8.908888   2.3051  -5.8308
1526   Topic5    16.876292                 mal    18.586308   2.3022  -5.0983
1848   Topic5     7.880020              parque     8.685648   2.3014 

In [ ]:
#Visualzing the models with pyLDAvis Using different MDS functions

pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer, mds='mmds')

In [145]:
#Visualzing the models with pyLDAvis Using different MDS functions
#Recomendado

pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer, mds='tsne')

C:\Users\Sara Palacios\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
3      35.082645        1       1   77.140495  -85.201279
0      31.891336        1       2  207.347244  -31.574324
4      17.124756        1       3  -31.592049   12.385936
1       9.608798        1       4   74.839226  105.217598
2       6.292465        1       5  215.774231  114.286545, topic_info=     Category         Freq              Term        Total  loglift  logprob
term                                                                       
3031  Default   130.000000           youtube   130.000000  30.0000  30.0000
514   Default  3155.000000                co  3155.000000  29.0000  29.0000
1369  Default    86.000000                gt    86.000000  28.0000  28.0000
1996  Default    58.000000             nueva    58.000000  27.0000  27.0000
1510  Default    77.000000       información    77.000000  26.0000  26.0000
1308  Default    56.000000    googlecolombia    56.000000  25.0000  25.0000
1294  Default   458.000000            google   458.000000  24.0000  24.0000
1447  Default    62.000000              hola    62.000000  23.0000  23.0000
791   Default    69.000000  desafíogoogleorg    69.000000  22.0000  22.0000
2769  Default    51.000000     thinkgooglela    51.000000  21.0000  21.0000
477   Default    30.000000            chrome    30.000000  20.0000  20.0000
1118  Default    37.000000          expertos    37.000000  19.0000  19.0000
1276  Default    36.000000             ghelp    36.000000  18.0000  18.0000
753   Default    41.000000             datos    41.000000  17.0000  17.0000
1330  Default    34.000000      googlemexico    34.000000  16.0000  16.0000
735   Default   145.000000              cómo   145.000000  15.0000  15.0000
1319  Default    75.000000     googleespanol    75.000000  14.0000  14.0000
1818  Default    75.000000             mejor    75.000000  13.0000  13.0000
2323  Default    39.000000         proyectos    39.000000  12.0000  12.0000
3026  Default    62.000000                ya    62.000000  11.0000  11.0000
559   Default    64.000000         compartir    64.000000  10.0000  10.0000
222   Default    80.000000           aprende    80.000000   9.0000   9.0000
3004  Default    56.000000               web    56.000000   8.0000   8.0000
294   Default    46.000000             ayuda    46.000000   7.0000   7.0000
1260  Default    61.000000     garagedigital    61.000000   6.0000   6.0000
384   Default    32.000000         búsquedas    32.000000   5.0000   5.0000
2743  Default    43.000000        tecnología    43.000000   4.0000   4.0000
1025  Default    31.000000            enlace    31.000000   3.0000   3.0000
1957  Default    53.000000           negocio    53.000000   2.0000   2.0000
532   Default    30.000000       colombianos    30.000000   1.0000   1.0000
...       ...          ...               ...          ...      ...      ...
1448   Topic5     6.366277          homenaje     7.235852   2.6378  -5.5716
1404   Topic5     5.739493             hacia     6.532312   2.6364  -5.6752
1687   Topic5     5.769462              link     6.570880   2.6357  -5.6700
704    Topic5     5.595008           cuentas     6.397201   2.6318  -5.7007
4      Topic5     5.338030                11     6.134425   2.6268  -5.7477
2650   Topic5     7.409080          software     8.542833   2.6234  -5.4199
2653   Topic5     4.958302            solano     5.726209   2.6218  -5.8215
1545   Topic5     4.750869           instala     5.505078   2.6185  -5.8643
653    Topic5     4.858532         convertir     5.656443   2.6138  -5.8418
2872   Topic5     4.701846                us     5.475023   2.6136  -5.8746
111    Topic5     5.262260           adsense     6.128537   2.6134  -5.7620
264    Topic5     8.835584           ataques    10.291772   2.6133  -5.2438
934    Topic5     4.856250      díademuertos     5.674803   2.6100  -5.8423
890    Topic5     5.221179        disfrútalo     6.1171

In [ ]:
#Future Work!!!
#Training Another LDA for visualization
# The vectorization is reused! Look above. 

lda_model = lda.LDA(n_topics = no_topics, n_iter=n_iter)
X_topics = lda_model.fit_transform(tf)

In [ ]:
print(X_topics.shape[0])

In [ ]:
print(lda_H.T.shape[0])

In [ ]:
# Threshold probability for topic assignment
threshold = 0.8

In [ ]:
#Threshold and plot

_idx = np.amax(X_topics, axis=1) > threshold  # idx of tweets that > threshold
_topics = X_topics[_idx]
_raw_tweet = np.array(original_tweets)[_idx]
_processed_tweet = np.array(tweets)[_idx]

In [ ]:
print(_topics)
#print(_raw_tweet)

In [ ]:
#number of tweets to show on the plot
num_example = 10

In [ ]:
#Reducing dimensions
# t-SNE: n -> 2D
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(_topics[:num_example])

In [ ]:
# find the most probable topic for each tweet
_lda_keys = []
for i, tweet in enumerate(_raw_tweet):
    _lda_keys += _topics[i].argmax(),

In [ ]:
print(len(_lda_keys))
print(_lda_keys[24])

In [ ]:
# generate random hex color
colormap = []
for i in range(X_topics.shape[1]):
    r = lambda: random.randint(0, 255)
    colormap += ('#%02X%02X%02X' % (r(), r(), r())),
colormap = np.array(colormap)

In [ ]:
# show topics and their top words
topic_summaries = []
topic_word = lda_model.topic_word_  # get the topic words
vocab = tf_feature_names

In [ ]:
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(no_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [ ]:
# use the coordinate of a random tweet as string topic string coordinate
topic_coord = np.empty((X_topics.shape[1], 2)) * np.nan
print(topic_coord[0])
print(tsne_lda)

In [ ]:
for topic_num in _lda_keys:
    if not np.isnan(topic_coord).any():
        print('entro')
        break
    topic_coord[topic_num] = tsne_lda[_lda_keys.index(topic_num)]

In [ ]:
for topic_num in _lda_keys:
    if not np.isnan(topic_coord).any():
        print('entro')
        break
    print(_lda_keys.index(topic_num))
    print(topic_num)
    print(tsne_lda[_lda_keys.index(topic_num)])

In [ ]:
# plot
title = "t-SNE visualization of LDA model trained on {} tweets, {} topics, " \
          "thresholding at {} topic probability, {} iter ({} data points and " \
          "top {} words)".format(len(tweets), no_topics, threshold,
                                 n_iter, num_example, no_top_words)
%matplotlib inline

In [ ]:
plot_lda = bp.figure(plot_width=900, plot_height=700,
                       title=title,
                       tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                       x_axis_type=None, y_axis_type=None, min_border=1)

In [ ]:
#data = dict(x=list(_raw_tweet[:num_example]),y=_lda_keys[:num_example])
data = {"tweet": _raw_tweet[:num_example],"topic_key": _lda_keys[:num_example]}
source = bp.ColumnDataSource(data = data)
print(data)

In [ ]:
plot_lda.scatter(  x=tsne_lda[:, 0], y=tsne_lda[:, 1],
                   color=colormap[_lda_keys][:num_example],
                   source = source)